In [4]:
try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
except:
    pass

import cv2
import numpy as np

cap = cv2.VideoCapture("/home/abhinav/Desktop/ENPM673/Project2/data/project_video.mp4")
K = np.array([[  1.15422732e+03,0.00000000e+00,6.71627794e+02],
              [  0.00000000e+00,1.14818221e+03,3.86046312e+02],
              [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])
dist = np.array([ -2.42565104e-01,-4.77893070e-02,  -1.31388084e-03,  -8.79107779e-05,
    2.20573263e-02])
while(True):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    undist = cv2.undistort(gray,K,dist)
    
    h = gray.shape[0]
    w = gray.shape[1]
    
    src = np.array([[w/2-50,h/2+110],[w/2+50,h/2+110],[w/2+200,h/2+250],[w/2-200,h/2+250]])
    dst = np.array([[300,300],[500,300],[500,500],[300,500]])
    H,flag = cv2.findHomography(src,dst)
    out= cv2.warpPerspective(undist,H,(w,h-200))
    blur = cv2.bilateralFilter(out,9,75,75)
    median = cv2.medianBlur(blur,5)
    mask = cv2.inRange(median,np.array([20,50,50]),np.array([30,255,255]))
    seg = cv2.bitwise_and(median,median,mask=mask)
    blur = cv2.bilateralFilter(seg,9,75,75)
#     canny = cv2.Canny(blur,75,255)


    
    
    
    
    cv2.imshow('video_out',blur)
#     cv2.imshow('video_out_undist',sobel)
    if cv2.waitKey(1)& 0xff==ord('q'):
        cv2.destroyAllWindows()
        break

In [4]:
gray[h/2+60:,(w/2-50):(w/2+50)]

array([[58, 54, 53, ..., 68, 58, 59],
       [57, 55, 52, ..., 63, 49, 52],
       [56, 58, 56, ..., 62, 47, 51],
       ...,
       [82, 87, 85, ..., 90, 93, 93],
       [80, 82, 82, ..., 86, 89, 89],
       [90, 90, 90, ..., 85, 87, 90]], dtype=uint8)